# Illustration of intrinsic dendritic filtering in dendritic stick and Hay model

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import neuron
import LFPy
import scipy.fftpack as ff
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots, simplify_axes, cmap_v_e

ns.load_mechs_from_folder(ns.cell_models_folder)
sigma = 0.3  # S/m

In [ ]:
frequencies = np.array([1, 200])  # Hz

grid_LFP_dict = {}
lat_LFP_dict = {}
imem_dict = {}
somav_dict = {}
syni_dict = {}
max_idx_dict = {}
tvec_dict = {}
color_dict = {frequencies[0]: 'gray',
              frequencies[1]: 'k',
              }

# Create a grid of measurement locations, in (um)
grid_x, grid_z = np.mgrid[-350:351:20, -400:1150:20]
grid_y = np.ones(grid_x.shape) * 0

grid_elec_params = {
    'sigma': sigma,      # extracellular conductivity
    'x': grid_x.flatten(),  # electrode positions
    'y': grid_y.flatten(),
    'z': grid_z.flatten(),
    'method': 'linesource'
}

lateral_elec_params = {
    'sigma': sigma,      # extracellular conductivity
    'x': np.cos(np.deg2rad(-30)) * np.linspace(2, 1000, 100),  # electrode positions
    'y': np.zeros(100),
    'z': np.sin(np.deg2rad(-30)) * np.linspace(2, 1000, 100),
    'method': 'linesource'
}

for freq in frequencies:

    tstop = 1000. / freq
    cell = ns.return_stick_cell(tstop, dt=tstop / 500)

    stim_idx = 0
    stim_params = {
                 'idx': stim_idx,
                 'record_current': True,
                 'syntype': 'SinSyn',
                 'del': 0.,
                 'dur': 1e9,
                 'pkamp': 0.1,
                 'freq': freq,
                 'weight': 1.0,  # not in use, but required for Synapse
                }

    syn = LFPy.Synapse(cell, **stim_params)
    cell.simulate(rec_vmem=True, rec_imem=True)

    max_idx = np.argmin(cell.imem[0, :])
    grid_electrode = LFPy.RecExtElectrode(cell, **grid_elec_params)
    grid_LFP =  grid_electrode.get_transformation_matrix() @ cell.imem[:, max_idx]

    lateral_electrode = LFPy.RecExtElectrode(cell, **lateral_elec_params)
    lateral_LFP = lateral_electrode.get_transformation_matrix() @ cell.imem[:, max_idx]

    imem_dict[freq] = cell.imem[:, max_idx]
    somav_dict[freq] = [cell.tvec.copy(), cell.vmem[0].copy()]
    grid_LFP_dict[freq] = 1000 * grid_LFP[:].reshape(grid_x.shape)
    lat_LFP_dict[freq] = 1000 * lateral_LFP[:]
    syni_dict[freq] = syn.i
    max_idx_dict[freq] = max_idx
    tvec_dict[freq] = cell.tvec
    
    del cell
    del syn
    del grid_LFP
    del lateral_electrode

In [ ]:
def make_WN_input(cell, freqs):
    """ White Noise input ala Linden 2010 is made """
    tot_ntsteps = round((cell.tstop - cell.tstart) / cell.dt + 1)
    I = np.zeros(tot_ntsteps)
    tvec = np.arange(tot_ntsteps) * cell.dt
    for freq in freqs:
        I += np.sin(2 * np.pi * freq * tvec/1000. + 2*np.pi*np.random.random())
    return I


def make_white_noise_stimuli(cell, input_idx, freqs, weight=None):

    input_scaling = 0.005
    np.random.seed(1234)
    input_array = input_scaling * (make_WN_input(cell, freqs))

    noise_vec = (neuron.h.Vector(input_array) if weight is None
                 else neuron.h.Vector(input_array * weight))

    i = 0
    syn = None
    for sec in cell.allseclist:
        for seg in sec:
            if i == input_idx:
                print("Input inserted in ", sec.name())
                syn = neuron.h.ISyn(seg.x, sec=sec)
            i += 1
    if syn is None:
        raise RuntimeError("Wrong stimuli index")
    syn.dur = 1E9
    syn.delay = 0
    noise_vec.play(syn._ref_amp, cell.dt)
    return cell, syn, noise_vec


### Also run simulation with white noise synapse (equal current amplitude at all frequencies)

In [ ]:
def simulate_white_noise_synapse(elec_x_pos, elec_z_pos, cell, freqs):

    stim_idx = 0
    cell, syn, noise_vec = make_white_noise_stimuli(cell, stim_idx, freqs)
    cell.simulate(rec_vmem=True, rec_imem=True)
    syn_i = np.array(noise_vec)

    lateral_elec_params = {
        'sigma': sigma,      # extracellular conductivity
        'x': elec_x_pos,  # electrode positions
        'y': np.zeros(2),
        'z': elec_z_pos,
        'method': 'linesource'
    }
    
    lateral_electrode = LFPy.RecExtElectrode(cell, **lateral_elec_params)
    lat_LFP = lateral_electrode.get_transformation_matrix() @ cell.imem
    freq_LFP, LFP_psd = ns.return_freq_and_psd(cell.tvec, lat_LFP)
    freq_syn, syn_i_psd = ns.return_freq_and_psd(cell.tvec, syn_i[-len(cell.tvec):])

    return freq_LFP, LFP_psd, syn_i_psd

elec_x_pos = np.array([10, 1000]) * np.cos(np.deg2rad(-30))
elec_z_pos = np.array([10, 1000]) * np.sin(np.deg2rad(-30))
tstop = 2**10  # ms
dt = 2**-6  # ms
num_tsteps = int(tstop / dt + 1)
timestep = dt/ 1000
sample_freq = ff.fftfreq(num_tsteps, d=timestep)
pidxs = np.where(sample_freq >= 0)
freqs = sample_freq[pidxs]
max_f_idx = np.argmin(np.abs(freqs - 1000))
freqs = freqs[:max_f_idx]

cell = ns.return_hay_cell(tstop, dt, make_passive=True)
freq_LFP_hay, LFP_psd_hay, syn_i_psd_hay = simulate_white_noise_synapse(elec_x_pos, elec_z_pos, cell, freqs)
del cell

cell = ns.return_stick_cell(tstop, dt)
freq_LFP_bns, LFP_psd_bns, syn_i_psd_bns = simulate_white_noise_synapse(elec_x_pos, elec_z_pos, cell, freqs)


### Plot results

In [ ]:
num = 11
levels = np.logspace(-2.5, 0, num=num)

scale_max = 10

levels_norm = scale_max * np.concatenate((-levels[::-1], levels))

colors_from_map = [cmap_v_e(i/(len(levels_norm) - 2))
                   for i in range(len(levels_norm) -1)]
colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)

fig = plt.figure(figsize=[6, 3])
fig.subplots_adjust(bottom=0.15, top=0.93, right=0.88, left=0.05, wspace=0.5,
                    hspace=0.05)

cax = fig.add_axes([0.38, 0.15, 0.01, 0.65], frameon=False, zorder=1000)

ax_dict = {}

ax_dec = fig.add_axes([0.57, 0.35, 0.15, 0.6], xticks=[1, 10, 100, 1000], zorder=10,
                      xticklabels=["1", "10", "100", "1000"],
                      xlabel="distance (µm)", 
                      xlim=[1, 1000], ylim=[1e-5, 2e2],
                      yscale="log", xscale="log")

ax_wn = fig.add_axes([0.83, 0.35, 0.15, 0.6], xticks=[1, 10, 100, ],
                      xticklabels=["1", "10", "100"],
                     yticks=[1e-3, 1e-2, 1e-1, 1e0, 1e1],
                      xlabel="frequency (Hz)", 
                      xlim=[1, 500], ylim=[1e-2, 2e0],
                      yscale="log", xscale="log")

ax_wn.set_ylabel("norm. PSD", labelpad=-0)
ax_dec.set_ylabel("µV²", labelpad=-0)

ax_dec.grid(True)
ax_wn.grid(True)

l1_bns, = ax_wn.plot(freq_LFP_bns, LFP_psd_bns[0] / LFP_psd_bns[0, 1], c='k', lw=2)
l2_bns, = ax_wn.plot(freq_LFP_bns, LFP_psd_bns[1] / LFP_psd_bns[1, 1], c='gray', lw=2)

l1_hay, = ax_wn.plot(freq_LFP_hay, LFP_psd_hay[0] / LFP_psd_hay[0, 1], c="cyan", lw=1, ls='-')
l2_hay, = ax_wn.plot(freq_LFP_hay, LFP_psd_hay[1] / LFP_psd_hay[1, 1], c='y', lw=1, ls='-')

fig.legend([l1_bns, l1_hay, l2_bns, l2_hay],
           [
            "stick at %d µm" % np.round(np.sqrt(elec_x_pos[0]**2 + elec_z_pos[0]**2)),
            "Hay at %d µm" % np.round(np.sqrt(elec_x_pos[0]**2 + elec_z_pos[0]**2)),
            "stick at %d µm" % np.round(np.sqrt(elec_x_pos[1]**2 + elec_z_pos[1]**2)),
            "Hay at %d µm" % np.round(np.sqrt(elec_x_pos[1]**2 + elec_z_pos[1]**2)),
           ],
           frameon=False, loc=[0.77, 0.01])

lines = []
line_names = []
for i, freq in enumerate(frequencies):

    imem = imem_dict[freq]
    LFP = grid_LFP_dict[freq]
    ax_LFP = fig.add_axes([0.02 + i * 0.2, 0.08, 0.17, 0.8], aspect=1,
                          frameon=False,  xticks=[], yticks=[],
                         ylim=[np.min(grid_electrode.z),
                               np.max(grid_electrode.z)],
                          xlim=[np.min(grid_electrode.x),
                                np.max(grid_electrode.x)])

    ax_stim = fig.add_axes([0.03 + i * 0.22, 0.86, 0.10, 0.1],
                           frameon=False, xticks=[], yticks=[])
    ax_stim.set_title("{:d} Hz".format(freq), pad=1)

    [ax_LFP.plot(cell.x[idx], cell.z[idx], lw=2, c='gray')
     for idx in range(cell.totnsegs)]

    ax_LFP.plot(cell.x[stim_idx].mean(), cell.z[stim_idx].mean(), 'o',
                c='y', ms=5, mec='k')

    ep_intervals = ax_LFP.contourf(grid_x, grid_z, LFP,
                                   zorder=-2, colors=colors_from_map,
                                   levels=levels_norm, extend='both')

    ax_LFP.contour(grid_x, grid_z, LFP, colors='k', linewidths=(1), zorder=-2,
                   levels=levels_norm)
    ax_LFP.plot(lateral_elec_params["x"], lateral_elec_params["z"], ls='--', c='gray', zorder=1)
    
    ax_stim.plot(tvec_dict[freq], syni_dict[freq], c='k')
    ax_stim.axvline(tvec_dict[freq][max_idx_dict[freq]], ls='--', c='gray')

    tstop = 1000. / freq

    if i == 1:
        ax_LFP.plot([-200, -200], [50, 250], c='k', lw=1)
        ax_LFP.text(-220, 100, "200 µm", ha='right')

    ax_stim.plot([tstop/2 + tstop/10, tstop + tstop/10], [-0.02, -0.02], lw=1, c='k')
    dur_marker = "{:1.1f} ms".format(tstop/2) if tstop/2 < 5 else "{:d} ms".format(int(tstop/2))
    ax_stim.text(tstop - tstop/2, -0.05, dur_marker, ha='left', va='top')
    ax_stim.plot([tstop * 1.1, tstop * 1.1], [-0.01, 0.09], lw=1, c='k')
    ax_stim.text(tstop * 1.15, 0.02, "0.1 nA", ha='left')
    mark_subplots(ax_stim, "AB"[i], xpos=-0.1, ypos=1)

    dist = np.sqrt(lateral_elec_params["x"]**2 + lateral_elec_params["z"]**2)
    l, = ax_dec.plot(dist, np.abs(lat_LFP_dict[freq])**2,
                     c='k', lw=1.5, ls="-:"[i])
    lines.append(l)
    line_names.append("{:d} Hz".format(freq))

mark_subplots(ax_dec, "C", ypos=1.02, xpos=-0.1)
mark_subplots(ax_wn, "D", ypos=1.02, xpos=-0.1)
fig.legend(lines, line_names, frameon=False, loc=(0.6, 0.11), ncol=1)
simplify_axes([ax_dec, ax_wn])
cbar = fig.colorbar(ep_intervals, cax=cax)

cbar.set_label('$V_\mathrm{e}$ (µV)', labelpad=0)
cbar.set_ticks([-10, -1, -0.1, 0.1, 1, 10])

plt.savefig("intrinsic_dend_filt.pdf")